In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp  /content/drive/MyDrive/kvasir_dataset.zip .

In [3]:
!unzip -qq kvasir_dataset.zip

In [4]:
!rm -rf kvasir_dataset/train/.ipynb_checkpoints

In [1]:
imagePaths = "kvasir_dataset/train/"
test_path = "kvasir_dataset/test/"

In [2]:
batch_size=64
img_height = 128
img_width = 128

In [3]:
import tensorflow as tf

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  imagePaths,
  #color_mode='grayscale',
  validation_split=0.15,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  #label_mode="categorical",
  batch_size=batch_size)

Found 15300 files belonging to 6 classes.
Using 13005 files for training.


In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  imagePaths, 
  #color_mode='grayscale',
  validation_split=0.15,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  #label_mode="categorical",
  batch_size=batch_size)

Found 15300 files belonging to 6 classes.
Using 2295 files for validation.


In [6]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_path,
  seed=123,
  image_size=(img_height, img_width),
  #label_mode="categorical",
  batch_size=batch_size)

Found 2700 files belonging to 6 classes.


In [7]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Layer, ReLU, concatenate
from tensorflow.keras.layers import Concatenate, Add
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Dropout, AvgPool2D

from tensorflow.keras.utils import plot_model

import warnings
warnings.simplefilter('ignore')

In [8]:
one = (1, 1)
two = (2, 2)
three = (3, 3)
five = (5, 5)
seven = (7, 7)
thirteen = (13, 13)

input_shape = (128, 128, 3)

In [9]:
class FireModule(object):
    """
    Fire Module computed as per the SqueezeNet paper
    """
    
    def __init__(self, layer_number: int, activation: str, kernel_initializer: str) -> None:
        """
        Constructor
        
        Arguments:
          layer_number       : Index of the Fire Module
          activation         : Activation to be used
          kernel_initializer : Kernel Weight Initialization technique
          
        Returns:
          None
        """
        
        self.layer_number = layer_number
        self.activation = activation
        self.kernel_initializer = kernel_initializer
        
    def build_module(self, fire_input: Layer) -> Layer:
        """
        Build the SqueezeNet
        
        Arguments:
          fire_input       : Input to Fire Module
          
        Returns:
          model            : SqueezeNet
        """
        
        global one, three, five
        
        output_size = 128 * (1 + (self.layer_number//2))
        
        squeeze_1x1_filters = 16 * (1 + (self.layer_number//2))
        expand_1x1_filters = expand_3x3_filters = output_size//2

        squeeze_1x1 = Conv2D(name=f'fire_{self.layer_number+2}_squeeze_1x1',
            filters=squeeze_1x1_filters, kernel_size=one, strides=1, padding='valid', activation=self.activation, 
            kernel_initializer=self.kernel_initializer)(fire_input)
        expand_1x1 = Conv2D(name=f'fire_{self.layer_number+2}_expand_1x1',
            filters=expand_1x1_filters, kernel_size=one, strides=1, padding='valid', activation=self.activation, 
            kernel_initializer=self.kernel_initializer)(squeeze_1x1)
        expand_3x3 = Conv2D(name=f'fire_{self.layer_number+2}_expand_3x3',
            filters=expand_3x3_filters, kernel_size=three, strides=1, padding='same', activation=self.activation, 
            kernel_initializer=self.kernel_initializer)(squeeze_1x1)

        fire = Concatenate(name=f'fire_{self.layer_number+2}')([expand_1x1, expand_3x3])
        
        return fire

In [10]:
class SqueezeNet(object):
    """
    SqueezeNet Architecture
    """
    
    def __init__(self, activation: str='relu', kernel_initializer: str='glorot_uniform') -> None:
        """
        Constructor
        
        Arguments:
          activation         : Activation to be used
          kernel_initializer : Kernel Weight Initialization technique
          
        Returns:
          None
        """
        
        self.activation = activation
        self.kernel_initializer = kernel_initializer
    
    def vanilla_model(self, input_shape: tuple=(128, 128, 3), n_classes: int=6) -> None:
        """
        Vanilla Implementation of SqueezeNet
        
        Arguments:
          input_shape         : Input Shape of the images
          n_classes           : Number of output classes
          
        Returns:
          None
        """
        
        inp = Input(shape=input_shape, name='Input')
        
        # Conv1 Layer
        conv_1 = Conv2D(name="Conv_1",
            filters=96, kernel_size=seven, strides=2, padding='same', activation=self.activation, kernel_initializer=self.kernel_initializer)(inp)
        maxpool_1 = MaxPool2D(name="MaxPool_1",
            pool_size=three, strides=2)(conv_1)
        
        # Fire 2-4
        fire_2 = FireModule(layer_number=0, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(maxpool_1)
        fire_3 = FireModule(layer_number=1, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(fire_2)
        fire_4 = FireModule(layer_number=2, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(fire_3)
        
        # Max Pool after Fire4 Module
        maxpool_2 = MaxPool2D(name="MaxPool_2",
            pool_size=three, strides=2)(fire_4)
        
        # Fire 5-8
        fire_5 = FireModule(layer_number=3, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(maxpool_2)
        fire_6 = FireModule(layer_number=4, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(fire_5)
        fire_7 = FireModule(layer_number=5, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(fire_6)
        fire_8 = FireModule(layer_number=6, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(fire_7)

        # Max Pool after Fire8 Module
        maxpool_3 = MaxPool2D(name="MaxPool_3",
            pool_size=three, strides=2)(fire_8)
        
        fire_9 = FireModule(layer_number=7, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(maxpool_3)
        
        # Dropout
        dropout = Dropout(0.5, name="Dropout")(fire_9)
        
        # Conv10 layer
        conv_10 = Conv2D(name="Conv_10",
            filters=1000, kernel_size=one, strides=1, padding='valid', activation=self.activation, kernel_initializer=self.kernel_initializer)(dropout)
        gap_11 = GlobalAveragePooling2D()(conv_10)

        
        if n_classes != 1000:
            # Add Dense(n_classes) and ouput == Dense layer
            out = Dense(n_classes, activation='softmax')(gap_11)
        else:
            out = gap_11
        
        self.model = Model(inputs=inp, outputs=out)
        
        
    def bypass_model(self, input_shape: tuple=(128, 128, 3), n_classes: int=6) -> None:
        """
        Residual Inspired Bypass Implementation of SqueezeNet
        
        Arguments:
          input_shape         : Input Shape of the images
          n_classes           : Number of output classes
          
        Returns:
          None
        """
        
        inp = Input(shape=input_shape, name='Input')
        
        # Conv1 Layer
        conv_1 = Conv2D(name="Conv_1",
            filters=96, kernel_size=seven, strides=2, padding='same', activation=self.activation, kernel_initializer=self.kernel_initializer)(inp)
        maxpool_1 = MaxPool2D(name="MaxPool_1",
            pool_size=three, strides=2)(conv_1)
        
        # Fire 2-4
        fire_2 = FireModule(layer_number=0, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(maxpool_1)
        fire_3 = FireModule(layer_number=1, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(fire_2)
        bypass_1 = Add(name="Bypass_1")([fire_2, fire_3])
        fire_4 = FireModule(layer_number=2, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(bypass_1)
        
        # Max Pool after Fire4 Module
        maxpool_2 = MaxPool2D(name="MaxPool_2",
            pool_size=three, strides=2)(fire_4)
        
        # Fire 5-8
        fire_5 = FireModule(layer_number=3, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(maxpool_2)
        bypass_2 = Add(name="Bypass_2")([maxpool_2, fire_5])
        fire_6 = FireModule(layer_number=4, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(bypass_2)
        fire_7 = FireModule(layer_number=5, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(fire_6)
        bypass_3 = Add(name="Bypass_3")([fire_6, fire_7])
        fire_8 = FireModule(layer_number=6, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(bypass_3)

        # Max Pool after Fire8 Module
        maxpool_3 = MaxPool2D(name="MaxPool_3",
            pool_size=three, strides=2)(fire_8)
        
        fire_9 = FireModule(layer_number=7, activation=self.activation, kernel_initializer=self.kernel_initializer).build_module(maxpool_3)
        bypass_4 = Add(name="Bypass_4")([maxpool_3, fire_9])
        
        # Dropout
        dropout = Dropout(0.5, name="Dropout")(bypass_4)
        
        # Conv10 layer
        conv_10 = Conv2D(name="Conv_10",
            filters=1000, kernel_size=one, strides=1, padding='valid', activation=self.activation, kernel_initializer=self.kernel_initializer)(dropout)
        gap_11 = GlobalAveragePooling2D()(conv_10)

        
        if n_classes != 1000:
            out = Dense(n_classes, activation='softmax')(gap_11)
        else:
            out = gap_11
        
        self.model = Model(inputs=inp, outputs=out)
    
    
    def build_model(self, input_shape: tuple=(128, 128, 3), n_classes: int=6, choice: str='vanilla') -> Model:
        """
        Build SqueezeNet
        
        Arguments:
          input_shape         : Input Shape of the images
          n_classes           : Number of output classes
          choice              : Type of architecture (vanilla/bypass)
        Returns:
          model               : SqueezeNet Model
        """
        
        if choice == "vanilla":
            self.vanilla_model(input_shape, n_classes)
        else:
            self.bypass_model(input_shape, n_classes)
        
        return self.model

In [11]:
snet = SqueezeNet()

model = snet.build_model(n_classes=6, choice='bypass')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv_1 (Conv2D)                (None, 64, 64, 96)   14208       ['Input[0][0]']                  
                                                                                                  
 MaxPool_1 (MaxPooling2D)       (None, 31, 31, 96)   0           ['Conv_1[0][0]']                 
                                                                                                  
 fire_2_squeeze_1x1 (Conv2D)    (None, 31, 31, 16)   1552        ['MaxPool_1[0][0]']          

In [12]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-4, clipvalue=0.5)
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer=opt, metrics=['accuracy',])

In [13]:
history = model.fit(train_ds, validation_data=val_ds, batch_size=64, epochs=25)

Epoch 1/25
204/204 [==============================] - 208s 945ms/step - loss: 1.0133 - accuracy: 0.5347 - val_loss: 0.7463 - val_accuracy: 0.6305
Epoch 2/25
204/204 [==============================] - 197s 937ms/step - loss: 0.7102 - accuracy: 0.6585 - val_loss: 0.6456 - val_accuracy: 0.6989
Epoch 3/25
204/204 [==============================] - 197s 936ms/step - loss: 0.6341 - accuracy: 0.6986 - val_loss: 0.6201 - val_accuracy: 0.7050
Epoch 4/25
204/204 [==============================] - 198s 935ms/step - loss: 0.6058 - accuracy: 0.7152 - val_loss: 0.6167 - val_accuracy: 0.7194
Epoch 5/25
204/204 [==============================] - 200s 951ms/step - loss: 0.5732 - accuracy: 0.7339 - val_loss: 0.5909 - val_accuracy: 0.7294
Epoch 6/25
204/204 [==============================] - 198s 941ms/step - loss: 0.5646 - accuracy: 0.7396 - val_loss: 0.5612 - val_accuracy: 0.7477
Epoch 7/25
204/204 [==============================] - 197s 934ms/step - loss: 0.5416 - accuracy: 0.7519 - val_loss: 0.6068 -

In [14]:
model.evaluate(test_ds)

43/43 [==============================] - 34s 638ms/step - loss: 0.3549 - accuracy: 0.8559


[0.354892760515213, 0.8559259176254272]

In [15]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [16]:
predictions = np.array([])
labels =  np.array([])
for x, y in test_ds:
  Y_pred=model.predict(x)
  y_prediction = np.argmax(Y_pred, axis=1)
  predictions = np.concatenate([predictions, y_prediction])
  labels = np.concatenate([labels, y.numpy()])

1/1 [==============================] - 0s 214ms/step


In [17]:
print('Confusion Matrix')
print(confusion_matrix(y_true=labels, y_pred=predictions))

Confusion Matrix
[[416  33   0   0   1   0]
 [ 89 361   0   0   0   0]
 [  0   0 446   4   0   0]
 [  0   0 102 334   8   6]
 [ 11   0   5  18 365  51]
 [  3   0   1  10  47 389]]


In [18]:
classes=['dyed-lifted-polyps','dyed-resection-margins', 'esophagitis','normal','polyps', 'ulcerative-colitis']
     
print('Classification Report')
target_names = classes
print(classification_report(y_true=labels, y_pred=predictions, target_names=target_names))

Classification Report
                        precision    recall  f1-score   support

    dyed-lifted-polyps       0.80      0.92      0.86       450
dyed-resection-margins       0.92      0.80      0.86       450
           esophagitis       0.81      0.99      0.89       450
                normal       0.91      0.74      0.82       450
                polyps       0.87      0.81      0.84       450
    ulcerative-colitis       0.87      0.86      0.87       450

              accuracy                           0.86      2700
             macro avg       0.86      0.86      0.85      2700
          weighted avg       0.86      0.86      0.85      2700



In [19]:
model.save("squeezenet_kvasir")

In [21]:
!zip -r squeezenet_kvasir.zip squeezenet_kvasir/

  adding: squeezenet_kvasir/ (stored 0%)
  adding: squeezenet_kvasir/saved_model.pb (deflated 90%)
  adding: squeezenet_kvasir/keras_metadata.pb (deflated 95%)
  adding: squeezenet_kvasir/variables/ (stored 0%)
  adding: squeezenet_kvasir/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: squeezenet_kvasir/variables/variables.index (deflated 76%)
  adding: squeezenet_kvasir/assets/ (stored 0%)
